# Processamento do Arquivo de Vacinação 

Este documento é utilizado para processar o arquivo **XLSX** disponibilizado pela **Secretaria de Estado da Saúde do estado de Sergipe** com os quantitativos da vacinação do estado. Foi utilizada a biblioteca **[pandas](https://pandas.pydata.org/)** para carregar e processar o arquivo com a finalidade de gerar um arquivo .csv mais adequado para o processo de visualização de dados. Um exemplo de visualização destes dados está no Painel COVID-19 disponível [neste link](https://covid19.data2learning.com).

Diferentes tipos de arquivos já foram compartilhados pela secretaria. Esse processamento se refere ao arquivo disponibilizado em **23/02/21** que tem a seguinte estrutura de colunas: 

* Municípios
* Trabalhadores de Saúde
    * População
        * TOTAL
    * 1ª Dose
        * Doses Enviadas
        * Doses Aplicadas
        * COB
    * 2ª Dose
        * Doses Enviadas
        * Doses Aplicadas
        * COB
* Pessoas com 60 anos ou mais e deficientes Institucionalizados
    * População
        * TOTAL
    * 1ª Dose
        * Doses Enviadas
        * Doses Aplicadas
        * COB
    * 2ª Dose
        * Doses Enviadas
        * Doses Aplicadas
        * COB
* Índio Aldeado
    * População
        * TOTAL
    * 1ª Dose
        * Doses Aplicadas
        * COB
    * 2ª Dose
        * Doses Aplicadas
        * COB
* Pessoas com 90 ou mais
    * Doses Enviadas
    * Doses Aplicadas
        * 1ª
        * COB
        * 2ª
        * COB
* TOTAL DE DOSES ENVIADAS
    * 1ª Dose
        * Doses Enviadas
        * Doses Aplicadas
        * COB
    * 2ª Dose
        * Doses Enviadas
        * Doses Aplicadas
        * COB
        
Totalizando 31 colunas com informações de população, doses enviadas, doses aplicadas e cobertura. 

## Etapas do Processamento

Para trabalhar melhor com as informações, foi preciso fazer um organização das informações disponbilizadas. Apesar do pandas lê facilmente arquivos do Excel, a estrutura em níveis das colunas no documento dificulta um pouco o trabalho no DataFrame gerado. Além disso, o documento possui algumas linhas com informações de título do documento como descrição do Governo, secretaria e coordenação responsável pelo documento. Como a idéia é gerar um arquivo .CSV que seja de fácil leitura, essas informações precisam ser removidas do documento.

As etapas do processamento foram: 

1. Definir uma nomeclatura para cada coluna para excluir a hierarquia criada no arquivo excel. 
2. Definir as colunas de interesse para geração do CSV. 
3. Carregar o XLSX e gerar o arquivo final.

Segue a execução das etapas. 

In [1]:
# Import da biblioteca pandas

import pandas as pd

### Nomeclatura das colunas

As colunas foram separadas em 4 grupos que corresponde a cada grupo apto para vacinação, conforme o arquivo do excel disponibilizado. São eles:

* Grupo 1: Trabalhadores da Saúde
* Grupo 2: Pessoas com 60 anos ou mais e deficientes Institucionalizados
* Grupo 3: Índio Aldeado
* Grupo 4: Pessoas com 90 ou mais

Com isso foram definidos  os nomes de cada coluna de acordo com o que estava disponível no arquivo original para cada grupo. Por exemplo, para o grupo 1 é informado a população, o quantitativo de doses envidas, o quantitativo de doses aplicadas e a cobertura. Essas últimas 3 informações para cada dose. Alguns grupos não possui todas essas informações. Por conta disso, a definição foi feita manualmente, como consta no código a seguir: 

In [2]:
# Lista com os nomes das colunas do documento excel.

list_columns = [
    'Municipio',
    'Grupo1_Populacao',
    'Grupo1_Dose1_Enviadas',
    'Grupo1_Dose1_Aplicadas',
    'Grupo1_Dose1_Cobertura',
    'Grupo1_Dose2_Enviadas',
    'Grupo1_Dose2_Aplicadas',
    'Grupo1_Dose2_Cobertura',
    'Grupo2_Populacao',
    'Grupo2_Dose1_Enviadas',
    'Grupo2_Dose1_Aplicadas',
    'Grupo2_Dose1_Cobertura',
    'Grupo2_Dose2_Enviadas',
    'Grupo2_Dose2_Aplicadas',
    'Grupo2_Dose2_Cobertura',
    'Grupo3_Populacao',
    'Grupo3_Dose1_Aplicadas',
    'Grupo3_Dose1_Cobertura',
    'Grupo3_Dose2_Aplicadas',
    'Grupo3_Dose2_Cobertura',
    'Grupo4_DosesEnviadas',
    'Grupo4_Doses1_Aplicadas',
    'Grupo4_Doses1_Cobertura',
    'Grupo4_Doses2_Aplicadas',
    'Grupo4_Doses2_Cobertura',
    'Total_Dose1_Enviadas',
    'Total_Dose1_Aplicadas',
    'Total_Dose1_Cobertura',
    'Total_Dose2_Enviadas',
    'Total_Dose2_Aplicadas',
    'Total_Dose2_Cobertura'
]

### Colunas de interesse 

Nem todas as colunas foram incluídas no CSV final. A coluna cobertura possui informação em %. Em alguns testes que foram feitos, o pandas não gerava essa coluna de forma adequada quando era carregado. Como essa é uma informação fácil de calcular pela relação entre as doses envidas e aplicadas, foi preferido retira-la do documento final. No processo de visualização essa informação pode ser facilmente calculada e exibida. Isso é feito no Painel linkado no início do documento. 

In [3]:
# Colunas definidas para o documento final

final_columns = [
    'Municipio',
    'Grupo1_Populacao',
    'Grupo1_Dose1_Enviadas',
    'Grupo1_Dose1_Aplicadas',
    'Grupo1_Dose2_Enviadas',
    'Grupo1_Dose2_Aplicadas',
    'Grupo2_Populacao',
    'Grupo2_Dose1_Enviadas',
    'Grupo2_Dose1_Aplicadas',
    'Grupo2_Dose2_Enviadas',
    'Grupo2_Dose2_Aplicadas',
    'Grupo3_Populacao',
    'Grupo3_Dose1_Aplicadas',
    'Grupo3_Dose2_Aplicadas',
    'Grupo4_DosesEnviadas',
    'Grupo4_Doses1_Aplicadas',
    'Grupo4_Doses2_Aplicadas',
    'Total_Dose1_Enviadas',
    'Total_Dose1_Aplicadas',
    'Total_Dose2_Enviadas',
    'Total_Dose2_Aplicadas',
]

### Carregar o XLSX e gerar o arquivo final

Os trechos de códigos a seguir servem para carregar o arquivo .xslx e gerar o arquivo CSV final. Como já definimos os nomes das colunas manualmente, podemos descartar as 8 primeiras linhas do documento que possui informações de título e nome das colunas de forma hierárquica. A explicação de cada etapa está nos comentários.

In [4]:
'''
Carrega o arquivo .xlsx descartando as 8 primeiras linhas (skiprows=8) que correspondem
as linhas de informações dos governos e títulos das colunas. O parâmetro names força que 
o dataframe gerado tenha os nomes das colunas definidas manualmente. 
''' 
data_final = pd.read_excel("data/Doses-aplicadas-23.02.xlsx", skiprows=8,names=list_columns)

'''
.head(-1) exclui a última linha do documento que possui o total de Sergipe. Essa informação
foi retirada porque é facilmente calcula a partir da soma de cada coluna. Isso pode ser feito
no momento de gerar as visualizações
''' 
data_final = data_final.head(-1)

'''
Gera o datframe com as colunas selecinadas para o documento final 
'''
data_final = data_final[final_columns]

'''
Alguns municípios possui valores em branco para algumas colunas. Quando o dataframe é carregado,
esses valores são reconhecidos como NULOS. O código a seguir substitui por 0.
'''
data_final = data_final.fillna(0)

'''
Força que as colunas que possui informações da vacinação seja do tipo inteiro.
final_columns[1:] pega da segunda coluna até a última, que são as colunas do tipo inteiro. 
'''
data_final[final_columns[1:]] = data_final[final_columns[1:]].astype(int)

'''
Visualiza o dataframe final 
''' 
data_final

,Municipio,Grupo1_Populacao,Grupo1_Dose1_Enviadas,Grupo1_Dose1_Aplicadas,Grupo1_Dose2_Enviadas,Grupo1_Dose2_Aplicadas,Grupo2_Populacao,Grupo2_Dose1_Enviadas,Grupo2_Dose1_Aplicadas,Grupo2_Dose2_Enviadas,...,Grupo3_Populacao,Grupo3_Dose1_Aplicadas,Grupo3_Dose2_Aplicadas,Grupo4_DosesEnviadas,Grupo4_Doses1_Aplicadas,Grupo4_Doses2_Aplicadas,Total_Dose1_Enviadas,Total_Dose1_Aplicadas,Total_Dose2_Enviadas,Total_Dose2_Aplicadas
0,Amparo de São Francisco,64,42,46,42,22,0,0,0,0,...,0,0,0,10,6,0,52,52,42,22
1,Aquidabã,397,257,258,257,160,17,17,17,17,...,0,0,0,98,83,0,372,358,274,176
2,Aracaju,32381,23639,14911,10939,8934,230,230,240,230,...,0,0,0,2061,2268,0,25930,17419,11169,9132
3,Arauá,219,142,142,142,58,0,0,0,0,...,0,0,0,30,30,0,172,172,142,58
4,Areia Branca,387,251,222,251,126,0,0,0,0,...,0,0,0,57,39,0,308,261,251,126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,Siriri,157,102,103,102,84,0,0,0,0,...,0,0,0,31,29,0,133,132,102,84
71,Telha,107,69,69,69,35,0,0,0,0,...,0,0,0,10,10,0,79,79,69,35
72,Tobias Barreto,1091,713,721,373,394,33,33,33,33,...,0,0,0,212,212,0,958,966,406,427
73,Tomar do Geru,161,104,110,104,73,0,0,0,0,...,0,0,0,42,23,0,146,133,104,73


O código a seguir gera o arquivo final no formato CSV. O atributo `index=False` força que o documento não tenha um índice por linha. Essa informação, neste caso, não é necessária para os processos de visualização.

In [5]:
data_final.to_csv("output/relatorio_vacina_sergipe.csv", index=False)

Desenvolvido por **@adolfoguimaraes**.

Qualquer dúvida sobre o código, só entrar em contato: 
* Twitter: https://twitter.com/adolfoguimaraes
* Instagram: https://instagram.com/profadolfoguimaraes
* GitHub: https://github.com/adolfoguimaraes

*Atenção: Esse trabalho é feito de forma totalmente independente e não tem nenhuma ligação com a secretaria que disponibiliza os dados.*